In [5]:
# import jupyter_black
import pandas as pd
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_confusion_matrix
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [6]:
# jupyter_black.load(lab=False, line_length=79)

# Задание 1
Для начала сделаем небольшую предобработку данных. Сколько всего в данных пропущенных значений?

In [7]:
df = pd.read_csv("data/AirPass.csv", index_col=0)

In [8]:
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [9]:
df.isnull().sum().sum()

310

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  object 
 2   Customer Type                      103904 non-null  object 
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  object 
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     1039

# Задание 2
Теперь давайте избавимся от найденных пропусков. Заполните их все медианными значениями.

In [11]:
df["Arrival Delay in Minutes"] = df["Arrival Delay in Minutes"].fillna(
    df["Arrival Delay in Minutes"].median()
)

In [12]:
round(df["Arrival Delay in Minutes"].mean(), 2)

15.13

# Задание 3
Проведём небольшой разведывательный анализ. Посмотрим, в каких категориях пассажиров превалировали удовлетворённые полетом клиенты.

Перекодируем часть бинарных признаков, чтобы использовать их при обучении:

In [13]:
df["satisfaction"] = df["satisfaction"].map(
    {"neutral or dissatisfied": 0, "satisfied": 1}
)

df["Gender"] = df["Gender"].map({"Male": 0, "Female": 1})

df["Customer Type"] = df["Customer Type"].map(
    {"Loyal Customer": 1, "disloyal Customer": 0}
)

df["Type of Travel"] = df["Type of Travel"].map(
    {"Personal Travel": 0, "Business travel": 1}
)

# Задание 4
Для остальных категориальных признаков создайте dummy-переменные. Сделайте это с помощью функции `get_dummies` из библиотеки Pandas, параметры не меняйте. Сколько теперь признаков в данных (включая целевую переменную)?

In [14]:
df = pd.get_dummies(df)

In [15]:
df.shape

(103904, 26)

# Задание 5
Мы практически добрались до обучения модели. Разбейте данные на обучающую и тестовую выборки в соотношении 80/20, параметр `random_state=26`.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns="satisfaction"),
    df["satisfaction"],
    test_size=0.2,
    random_state=26,
)

In [17]:
y_test.shape

(20781,)

# Задание 6
Теперь нам необходимо реализовать масштабирование данных. Для этого обучите на обучающей выборке метод `StandardScaler` и с помощью него преобразуйте и обучающую, и тестовую выборки. Не забудьте, что целевую переменную обрабатывать не нужно.

In [18]:
scaler = StandardScaler()

In [19]:
X_train = scaler.fit_transform(X_train)

In [20]:
X_test = scaler.transform(X_test)

In [21]:
round(X_test[0][0], 2)

0.94

# Задание 11
Обучите алгоритм CatBoost. Как и XGBoost, будем обучать его с настройками по умолчанию.

In [22]:
clf = CatBoostClassifier(verbose=False, random_state=42)

In [23]:
clf.fit(X_train, y_train)

In [24]:
y_pred = clf.predict(X_test)

In [25]:
round(f1_score(y_pred, y_test), 3)

0.96

# Задание 12
Выведите матрицу ошибок для алгоритма, который получил наилучшие показатели качества модели на обучающей выборке (будем считать, что оцениваем по `f1_score`).

In [26]:
get_confusion_matrix(clf, Pool(X_train, y_train))

array([[46665.,   541.],
       [ 1249., 34668.]])